In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as m
from scipy.stats import linregress # used to get the linear parameters from the results

In [ ]:
# Function will compute the strong error for the implicit EM approximation of the Lamperti transformation
# The Milstein approximation of the SABR model will be used as the reference solution to calculate strong error
def Implicit(T, N, sig_0, M, alpha, Y_0, rho, theta):
    # sig_0 = sigma(0)
    dt = T/N # step size
    Milstein_N = 2**18 # Number of steps for reference solution
    Y_n = np.ones(M)*Y_0 # array to hold values for Y at t for each monte carlo simulation
    sig_t = np.ones(M)*sig_0 # array to hold values for sigma at t for each monte carlo simulation
    F_0 = (Y_0 * (1 - theta))**(1/(1 - theta)) # convert Y(0) to F(0)
    Mil = np.ones(M)*F_0 # array to hold values for Milstein at t for each monte carlo simulation
    A = np.array([alpha*rho, alpha*np.sqrt(1 - rho**2)]) # Cholesky factor
    dt_mil = T/Milstein_N # step size for reference solution
    B = np.zeros(M) # empty array to hold cumulative Brownian motion
    B1_Y = np.zeros(M) # empty array to hold Brownian motion
    for i in range(Milstein_N):
        Z = np.sqrt(dt_mil) * np.random.normal(0, 1, (2, M)) # calculate Brownian increment for B1 and B2
        B += Z[0,:] # cumulate the Brownian increment for B1
        B2 = A[0]*Z[0,:] + A[1]*Z[1,:]  # Brownian motion for B2 in the reference solution
        B1 = Z[0, :] # Brownian motion for B1 in the reference solution
        sig_t = sig_t*np.exp((-alpha**2 / 2)*dt_mil + B2) # sigma(t)
        Mil = Mil + sig_t*B1*np.abs(Mil)**theta +\
            0.5*(sig_t**2)*theta*(B1**2 - dt_mil)*np.abs(Mil)**(2*theta - 1) # Milstein approximation
        
        # to keep on the same path, Yn will be calculated when the below condition is true
        if (i+1)%(Milstein_N//N) == 0:
            Y_n = ((Y_n + sig_t*(B-B1_Y))/2) + \
                np.sqrt((((Y_n + sig_t*(B-B1_Y))**2)/4) - (dt * theta * sig_t**2) / (2 * (1 - theta))) # Yn
            B1_Y = np.copy(B) # Store the cumulative Brownian motion for B1
    
    F_n = (Y_n * (1 - theta))**(1/(1 - theta)) # convet Yn to Fn
    return F_n, Mil # Return the values when n=N

### Alpha affect on strong order of convergence

In [ ]:
T = 1 # time
rho = 0.25 # correlation between B1 and B2
M=10**3 # No. of Monte Carlo simulations

In [ ]:
alpha = np.array([0.01, 0.0125, 0.025, 0.0375, 0.05, 0.075, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]) # values of alpha that will be evaluated
thetaSim = np.array([0.5, 0.6, 0.7, 0.8, 0.9]) # range theta that will be evaluated
Nsteps = np.array([ 2**6, 2**8, 2**9, 2**10, 2**12]) # Range of step sizes
Alpha_error = np.zeros((len(alpha), len(Nsteps), len(thetaSim)))
for k in range(len(alpha)):
    for i in range(len(Nsteps)):
        for j in range(len(thetaSim)):
            F_T, Mil= Implicit(T=T, N=Nsteps[i], M=M, sig_0=0.05, alpha=alpha[k], Y_0=10, rho=rho, theta=thetaSim[j])
            Alpha_error[k,i,j] = np.mean(np.abs(Mil - F_T)) # Strong error

In [ ]:
# calculation for the strong order of convergence, gamma, and C
Dt = T / Nsteps
gamma_array = np.zeros((len(alpha), len(thetaSim)))
for j in range(len(alpha)):         
    for i in range(len(thetaSim)):
        Mil_gamma, Mil_C = linregress(np.log(Dt), np.log(Alpha_error[j, :,i]))[0:2]
        C_Mil = m.exp(Mil_C)
        gamma_array[j, i] = Mil_gamma
        print('For the Milstein method, when theta={}, , alpha = {}, gamma = {} and C = {}\n'.format(round(thetaSim[i],1), alpha[j], Mil_gamma, C_Mil))

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(alpha, gamma[:, 4],'x', linestyle='--', label='theta={}'.format(round(thetaSim[4], 1)))
plt.plot(alpha, gamma[:, 5],'d', linestyle='--', label='theta={}'.format(round(thetaSim[5], 1)))
plt.plot(alpha, gamma[:, 6],'o', linestyle='--', label='theta={}'.format(round(thetaSim[6], 1)))
plt.plot(alpha, gamma[:, 7],'s', linestyle='--', label='theta={}'.format(round(thetaSim[7], 1)))
plt.plot(alpha, gamma[:, 8],'*', linestyle='--', label='theta={}'.format(round(thetaSim[8], 1)))
plt.legend(fontsize=14, ncol=2)
plt.xlabel(r'$\alpha$', fontsize=22)
plt.ylabel('Order of convergence', fontsize=18)
plt.rc('xtick',labelsize=16)
plt.rc('ytick',labelsize=16)
plt.grid()
# plt.savefig('alpha_transformation.jpg')
plt.show()

### Rho affect on strong order of convergence

In [ ]:
T = 1 # time
alpha = 0.5 # alpha coefficient in the SABR model
M=10**3 # No. of Monte Carlo simulations

In [ ]:
rho = np.array([-1, -0.67, -0.33, 0, 0.33, 0.67, 1]) # rho values that will be evaluated
thetaSim = np.array([0.5, 0.6, 0.7, 0.8, 0.9]) # range theta that will be evaluated
Nsteps = np.array([ 2**6, 2**8, 2**9, 2**10, 2**12]) # Range of step sizes
rho_error = np.zeros((len(rho), len(Nsteps), len(thetaSim)))
for k in range(len(rho)):
    for i in range(len(Nsteps)):
        for j in range(len(thetaSim)):
            F_T, Mil= Implicit(T=T, N=Nsteps[i], M=M, sig_0=0.2, alpha=alpha, Y_0=10, rho=rho[k], theta=thetaSim[j])
            rho_error[k,i,j] = np.mean(np.abs(Mil - F_T)) # Strong error for Milstein

In [ ]:
# calculation for the strong order of convergence, gamma, and C
Dt = T / Nsteps
gamma_array_rho = np.zeros((len(rho), len(thetaSim)))
for j in range(len(rho)):         
    for i in range(len(thetaSim)):
        Mil_gamma, Mil_C = linregress(np.log(Dt), np.log(rho_error[j, :,i]))[0:2]
        C_Mil = m.exp(Mil_C)
        gamma_array_rho[j, i] = Mil_gamma
        print('For the Milstein method, when theta={}, , rho = {}, gamma = {} and C = {}\n'.format(round(thetaSim[i],1), rho[j], Mil_gamma, C_Mil))

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(rho, gamma_array_rho[:, 4],'x', linestyle='--', label='theta={}'.format(round(thetaSim[4], 1)))
plt.plot(rho, gamma_array_rho[:, 5],'d', linestyle='--', label='theta={}'.format(round(thetaSim[5], 1)))
plt.plot(rho, gamma_array_rho[:, 6],'o', linestyle='--', label='theta={}'.format(round(thetaSim[6], 1)))
plt.plot(rho, gamma_array_rho[:, 7],'s', linestyle='--', label='theta={}'.format(round(thetaSim[7], 1)))
plt.plot(rho, gamma_array_rho[:, 8],'*', linestyle='--', label='theta={}'.format(round(thetaSim[8], 1)))
plt.legend(fontsize=14, ncol=2)
plt.xlabel(r'$\rho$', fontsize=22)
plt.ylabel('Order of convergence', fontsize=18)
plt.rc('xtick',labelsize=16)
plt.rc('ytick',labelsize=16)
plt.grid()
# plt.savefig('rho_transformation.jpg')
plt.show()

### sigma(0) affect on strong order of convergence

In [ ]:
T = 1 # time
alpha = 0.01 # alpha coefficient in the SABR model
rho = 0.25 # correlation between B1 and B2
M=10**3 # No. of Monte Carlo Simulations

In [ ]:
sig_0 = np.array([0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]) # range of sigma(0) values to be evaluated
thetaSim = np.array([0.5, 0.6, 0.7, 0.8, 0.9]) # range theta that will be evaluated
Nsteps = np.array([ 2**6, 2**8, 2**9, 2**10, 2**12]) # Range of step sizes
sig_0_error = np.zeros((len(sig_0), len(Nsteps), len(thetaSim)))
for k in range(len(sig_0)):
    for i in range(len(Nsteps)):
        for j in range(len(thetaSim)):
            F_T, Mil= Implicit(T=T, N=Nsteps[i], M=M, sig_0=sig_0[k], alpha=alpha, Y_0=10, rho=rho, theta=thetaSim[j])
            sig_0_error[k,i,j] = np.mean(np.abs(Mil - F_T)) # Strong error

In [ ]:
# calculation for the strong order of convergence, gamma, and C
Dt = T / Nsteps
gamma_array_sig_0 = np.zeros((len(sig_0), len(thetaSim)))
for j in range(len(sig_0)):         
    for i in range(len(thetaSim)):
        Mil_gamma, Mil_C = linregress(np.log(Dt), np.log(sig_0_error[j, :,i]))[0:2]
        C_Mil = m.exp(Mil_C)
        gamma_array_sig_0[j, i] = Mil_gamma
        print('For the Milstein method, when theta={}, , sig_0 = {}, gamma = {} and C = {}\n'.format(round(thetaSim[i],1), sig_0[j], Mil_gamma, C_Mil))

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(sig_0, gamma[:, 4],'x', linestyle='--', label='theta={}'.format(round(thetaSim[4], 1)))
plt.plot(sig_0, gamma[:, 5],'d', linestyle='--', label='theta={}'.format(round(thetaSim[5], 1)))
plt.plot(sig_0, gamma[:, 6],'o', linestyle='--', label='theta={}'.format(round(thetaSim[6], 1)))
plt.plot(sig_0, gamma[:, 7],'s', linestyle='--', label='theta={}'.format(round(thetaSim[7], 1)))
plt.plot(sig_0, gamma[:, 8],'*', linestyle='--', label='theta={}'.format(round(thetaSim[8], 1)))
plt.legend(fontsize=14, ncol=2)
plt.xlabel(r'$\sigma (0)$', fontsize=22)
plt.ylabel('Order of convergence', fontsize=18)
plt.rc('xtick',labelsize=16)
plt.rc('ytick',labelsize=16)
plt.grid()
# plt.savefig('sigma_0_transformation.jpg')
plt.show()